In [2]:
using Laplacians

In [1]:
1+2

3

In [3]:
type IJV{Tv,Ti}
    n::Ti
    nnz::Ti
    i::Array{Ti,1}
    j::Array{Ti,1}
    v::Array{Tv,1}
end


In [4]:
function IJV{Tv,Ti}(A::SparseMatrixCSC{Tv, Ti}) 
    (ai,aj,av) = findnz(A)
    IJV{Tv,Ti}(A.n, nnz(A), ai, aj, av)
end


IJV{Tv,Ti}

In [5]:
a = sparse(rand(5,5))
ijv = IJV(a)

IJV{Float64,Int64}(5,25,[1,2,3,4,5,1,2,3,4,5  …  1,2,3,4,5,1,2,3,4,5],[1,1,1,1,1,2,2,2,2,2  …  4,4,4,4,4,5,5,5,5,5],[0.78328,0.244047,0.737132,0.407148,0.968147,0.705672,0.877284,0.538385,0.197892,0.287365  …  0.00362295,0.567457,0.885835,0.962108,0.442368,0.874936,0.85636,0.83223,0.505683,0.484684])

In [6]:
import Base.SparseMatrix.sparse
sparse(IJV) = sparse(IJV.i, IJV.j, IJV.v, IJV.n, IJV.n)
sparse(ijv)
sum(abs(a-sparse(ijv)))

0.0

In [7]:
function generalizedNecklace{Tv, Ti}(A::IJV{Tv, Ti}, H::IJV{Tv, Ti}, k::Int64)

  # these are square matrices
  n = A.n
  m = H.n

  newI = Ti[]
  newJ = Ti[]
  newW = Tv[]

  # duplicate the vertices in A so that each vertex in H corresponds to a copy of A
  for i in 1:m
    newI = append!(newI, A.i + n * (i - 1))
    newJ = append!(newJ, A.j + n * (i - 1))
    newW = append!(newW, A.v)
  end

  # for each edge in H, add k random edges between two corresponding components
  # multiedges will be concatenated to a single edge with higher cost
  for i in 1:H.nnz
    u = H.i[i]
    v = H.j[i]

    if (u < v)
      #component x is from 1 + (x - 1) * n to n + (x - 1) * n
      for edgeToAdd in 1:k
                r1 = rand(1:n)
                r2 = rand(1:n)
        newU = r1 + n * (u - 1)
        newV = r2 + n * (v - 1)
        append!(newI, [newU, newV])
        append!(newJ, [newV, newU])
        append!(newW, [1, 1])
      end
    end
  end

    return IJV{Tv, Ti}(n*m, length(newI), newI, newJ, newW)
end # generalizedNecklace


generalizedNecklace (generic function with 1 method)

In [8]:
function generalizedNecklace2{Tv, Ti}(A::IJV{Tv, Ti}, H::IJV{Tv, Ti}, k::Int64)

    # these are square matrices
    n = A.n
    m = H.n

    newI = Array(Ti, m*A.nnz)
    newJ = Array(Ti, m*A.nnz)
    newW = Array(Tv, m*A.nnz)
    
    ptr = 1
    for i in 1:m
        for j in 1:A.nnz
            newI[ptr] = A.i[j] + n*(i-1)
            newJ[ptr] = A.j[j] + n*(i-1)
            newW[ptr] = A.v[j] 
            ptr += 1
        end
    end

    

  # for each edge in H, add k random edges between two corresponding components
  # multiedges will be concatenated to a single edge with higher cost
    
    # this is not for efficiency, but for backwards compatability
    rnd = rand(1:n, k*H.nnz)
    nextI = rnd[1:2:end]
    nextJ = rnd[2:2:end]
    
    nextW = ones(Tv, k*H.nnz)
    
    ptr = 1
    for i in 1:H.nnz
        u = H.i[i]
        v = H.j[i]

        if (u < v)
            #component x is from 1 + (x - 1) * n to n + (x - 1) * n
            for edgeToAdd in 1:k
                nextI[ptr] += n * (u - 1)
                nextJ[ptr] += n * (v - 1)
                ptr += 1
            end
        end
    end
  
    newI = [newI; nextI; nextJ]
    newJ = [newJ; nextJ; nextI]
    newW = [newW; nextW]
    
    return IJV{Tv, Ti}(n*m, length(newI), newI, newJ, newW)
end # generalizedNecklace



generalizedNecklace2 (generic function with 1 method)

In [9]:
srand(1)
@show r = rand(1:10,11)
@show r[1:2:end]
@show r[2:2:end]

r = rand(1:10,11) = [9,7,10,10,4,1,3,4,9,2,1]
r[1:2:end] = [9,10,4,3,9,1]
r[2:2:end] = [7,10,1,4,2]


5-element Array{Int64,1}:
  7
 10
  1
  4
  2

In [84]:
srand(1)
for i in 1:10
    @show rand(1:10,1)
end

rand(1:10,1) = [9]
rand(1:10,1) = [7]
rand(1:10,1) = [10]
rand(1:10,1) = [10]
rand(1:10,1) = [4]
rand(1:10,1) = [1]
rand(1:10,1) = [3]
rand(1:10,1) = [4]
rand(1:10,1) = [9]
rand(1:10,1) = [2]


In [62]:
sizeof(x)

LoadError: LoadError: type does not have a canonical binary representation
while loading In[62], in expression starting on line 1

In [23]:
a = randRegular(2000,3)
ijv = IJV(a)
srand(1)
@time ijv3 = generalizedNecklace2(ijv,ijv,2)

srand(1)
@time ijv2 = generalizedNecklace(ijv,ijv,2)
srand(1)
@time a2 = Laplacians.generalizedNecklace(a,a,2)
sum(abs(sparse(ijv2) - a2))
sum(abs(sparse(ijv3) - a2))

  0.451904 seconds (27 allocations: 549.867 MB, 37.94% gc time)
  0.565221 seconds (50.04 k allocations: 470.597 MB, 47.76% gc time)
  1.498872 seconds (50.07 k allocations: 990.038 MB, 19.53% gc time)


0.0

In [96]:
sizeof(ijv2.nnz)

8

In [94]:
ijv3.nnz

64

In [27]:
ijv2

IJV{Float64,Int64}(16,32,[2,4,1,3,2,4,1,3,6,8  …  9,11,14,16,13,15,14,16,13,15],[1,1,2,2,3,3,4,4,5,5  …  12,12,13,13,14,14,15,15,16,16],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])

In [19]:
ijv2.nnz

32

In [20]:
nnz(a2)

40

In [10]:
Laplacians.generalizedNecklace(a,a,1)

100x100 sparse matrix with 264 Float64 entries:
	[2  ,   1]  =  1.0
	[3  ,   1]  =  1.0
	[4  ,   1]  =  1.0
	[1  ,   2]  =  1.0
	[8  ,   2]  =  2.0
	[30 ,   2]  =  1.0
	[1  ,   3]  =  1.0
	[7  ,   3]  =  2.0
	[1  ,   4]  =  1.0
	[9  ,   4]  =  1.0
	⋮
	[98 ,  96]  =  1.0
	[99 ,  96]  =  1.0
	[93 ,  97]  =  2.0
	[96 ,  97]  =  1.0
	[92 ,  98]  =  2.0
	[96 ,  98]  =  1.0
	[94 ,  99]  =  1.0
	[95 ,  99]  =  1.0
	[96 ,  99]  =  1.0
	[94 , 100]  =  1.0
	[95 , 100]  =  2.0

In [4]:
a.m

5

In [5]:
fieldnames(a)

5-element Array{Symbol,1}:
 :m     
 :n     
 :colptr
 :rowval
 :nzval 